In [15]:
import pandas as pd
df = pd.read_csv('https://query.data.world/s/yVjCltW5yFnDbwy0lmj7doFPPTB9K9')

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [59]:
import datetime 

In [23]:
for x in df.columns.values: # check out data headers
    print(x)

CMPLNT_NUM
CMPLNT_FR_DT
CMPLNT_FR_TM
CMPLNT_TO_DT
CMPLNT_TO_TM
RPT_DT
KY_CD
OFNS_DESC
PD_CD
PD_DESC
CRM_ATPT_CPTD_CD
LAW_CAT_CD
JURIS_DESC
BORO_NM
ADDR_PCT_CD
LOC_OF_OCCUR_DESC
PREM_TYP_DESC
PARKS_NM
HADEVELOPT
X_COORD_CD
Y_COORD_CD
Latitude
Longitude
Lat_Lon


In [94]:
for x in df['OFNS_DESC'].unique(): # view unique crime types 
    print(x)

FORGERY
MURDER & NON-NEGL. MANSLAUGHTER
DANGEROUS DRUGS
ASSAULT 3 & RELATED OFFENSES
FELONY ASSAULT
DANGEROUS WEAPONS
PETIT LARCENY
GRAND LARCENY
ROBBERY
OFFENSES AGAINST PUBLIC ADMINI
CRIMINAL MISCHIEF & RELATED OF
RAPE
INTOXICATED & IMPAIRED DRIVING
HARRASSMENT 2
SEX CRIMES
BURGLARY
CRIMINAL TRESPASS
MISCELLANEOUS PENAL LAW
VEHICLE AND TRAFFIC LAWS
OFF. AGNST PUB ORD SENSBLTY &
FRAUDS
GRAND LARCENY OF MOTOR VEHICLE
OFFENSES INVOLVING FRAUD
OFFENSES AGAINST THE PERSON
FRAUDULENT ACCOSTING
OTHER OFFENSES RELATED TO THEF
GAMBLING
ARSON
POSSESSION OF STOLEN PROPERTY
UNAUTHORIZED USE OF A VEHICLE
THEFT-FRAUD
DISORDERLY CONDUCT
nan
ADMINISTRATIVE CODE
CHILD ABANDONMENT/NON SUPPORT
OTHER STATE LAWS (NON PENAL LA
NYS LAWS-UNCLASSIFIED FELONY
BURGLAR'S TOOLS
THEFT OF SERVICES
PETIT LARCENY OF MOTOR VEHICLE
ALCOHOLIC BEVERAGE CONTROL LAW
AGRICULTURE & MRKTS LAW-UNCLASSIFIED
ENDAN WELFARE INCOMP
KIDNAPPING & RELATED OFFENSES
DISRUPTION OF A RELIGIOUS SERV
PROSTITUTION & RELATED OFFENSES
JOSTLIN

In [186]:
# create list of crimes that we want for our visualization
crimes = ['MURDER & NON-NEGL. MANSLAUGHTER', 'ASSAULT 3 & RELATED OFFENSES', 'FELONY ASSAULT', 'ROBBERY', 'HARRASSMENT 2', 'HOMICIDE-NEGLIGENT,UNCLASSIFIE', 'KIDNAPPING', 'KIDNAPPING AND RELATED OFFENSES', 'HOMICIDE-NEGLIGENT-VEHICLE', 'KIDNAPPING & RELATED OFFENSES', 'PETIT LARCENY OF MOTOR VEHICLE', 'PETIT LARCENY', 'GRAND LARCENY', 'GRAND LARCENY OF MOTOR VEHICLE']
# create list of locations we want to exclude
private_loc = ['HOTEL/MOTEL', 'RESIDENCE - PUBLIC HOUSING', 'RESIDENCE - APT. HOUSE', 'RESIDENCE-HOUSE', 'PARKING LOT/GARAGE (PRIVATE)']

In [187]:
df1 = df[df['OFNS_DESC'].isin(crimes)] # get only crimes we're interested in 
df1 = df1[~df1['PREM_TYP_DESC'].isin(private_loc)] # get only public location crimes
# drop nulls for specified columns
df1 = df1.dropna(subset = ['CMPLNT_FR_DT', 'CMPLNT_FR_TM', 'BORO_NM', 'ADDR_PCT_CD'])

In [188]:
print('Percent Remaining: {}'.format(100*df1.shape[0]/df.shape[0]))

Percent Remaining: 36.19458488504158


In [189]:
def getSeason(date):
    """
    Funciton to convert the date to the season.
    
    Input: date (datetime.datetime obj)
    """
    y = date.year 
    spring = datetime.datetime(y, 3, 20)
    summer = datetime.datetime(y, 6, 21)
    fall = datetime.datetime(y, 9, 22)
    winter = datetime.datetime(y, 12, 21)
    if date >= spring and date < summer:
        return "SPRING"
    if date >= summer and date < fall:
        return "SUMMER"
    if date >= fall and date < winter:
        return "FALL"
    if date >= winter or date < spring:
        return "WINTER"
    return -1

In [190]:
# create season column
df1['SEASON'] = df1['CMPLNT_FR_DT'].map(lambda x: getSeason(datetime.datetime.strptime(x, '%m/%d/%Y')))
# replace weird crime names to something more digestable
d = {'ASSAULT 3 & RELATED OFFENSES': 'ASSAULT', 
     'FELONY ASSAULT': 'ASSAULT', 
     'HARRASSMENT 2': 'HARRASSMENT',
    'HOMICIDE-NEGLIGENT-VEHICLE': 'NEGLIGENT HOMICIDE',
    'HOMICIDE-NEGLIGENT,UNCLASSIFIE': 'NEGLIGENT HOMICIDE',
    'KIDNAPPING & RELATED OFFENSES': 'KIDNAPPING',
    'KIDNAPPING AND RELATED OFFENSES': 'KIDNAPPING',
     'MURDER & NON-NEGL. MANSLAUGHTER': 'MURDER'
    }
df1['OFNS_DESC'] = df1['OFNS_DESC'].replace(d)

In [191]:
# check out the new crime names
for i in df1['OFNS_DESC'].unique():
    print(i)

MURDER
ASSAULT
PETIT LARCENY
GRAND LARCENY
ROBBERY
HARRASSMENT
GRAND LARCENY OF MOTOR VEHICLE
PETIT LARCENY OF MOTOR VEHICLE
KIDNAPPING
NEGLIGENT HOMICIDE


In [192]:
# import populations of precinct 
# source https://fusiontables.google.com/DataSource?dsrcid=767562#rows:id=1
pdf = pd.read_csv('/home/jakesell/Documents/datachall/NYC_Blocks_2010CensusData_Plus_Precincts.csv')

In [193]:
# create series that relates pricinct to its total population
s = pdf.groupby('precinct')['P0010001'].sum()

In [194]:
def getPop(p):
    """
    Get the population of a precinct based on precinct number
    
    Input: p (precinct, int)
    Output: total population (int)
    """
    try:
        return s[p]
    except KeyError:
        return 0
    
# add column with a row's precinct population
df1['precint_pop'] = df1['ADDR_PCT_CD'].map(lambda x: getPop(x))

In [195]:
df1.to_csv('CleanedCrimeData0.csv') # export

In [196]:
df1.head()

,CMPLNT_NUM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,RPT_DT,KY_CD,OFNS_DESC,PD_CD,PD_DESC,...,PREM_TYP_DESC,PARKS_NM,HADEVELOPT,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon,SEASON,precint_pop
1,153401121,12/31/2015,23:36:00,NaN,NaN,12/31/2015,101,MURDER,NaN,NaN,...,NaN,NaN,NaN,1043991.0,193406.0,40.697338,-73.784557,"(40.697338138, -73.784556739)",WINTER,105803
4,641637920,12/31/2015,23:25:00,12/31/2015,23:30:00,12/31/2015,344,ASSAULT,101.0,ASSAULT 3,...,OTHER,NaN,NaN,987606.0,208148.0,40.738002,-73.987891,"(40.7380024, -73.98789129)",WINTER,93640
5,365661343,12/31/2015,23:18:00,12/31/2015,23:25:00,12/31/2015,106,ASSAULT,109.0,"ASSAULT 2,1,UNCLASSIFIED",...,DRUG STORE,NaN,NaN,996149.0,181562.0,40.665023,-73.957111,"(40.665022689, -73.957110763)",WINTER,98429
9,415095955,12/31/2015,23:10:00,12/31/2015,23:10:00,12/31/2015,341,PETIT LARCENY,338.0,"LARCENY,PETIT FROM BUILDING,UN",...,DRUG STORE,NaN,NaN,994327.0,218211.0,40.765618,-73.963623,"(40.765617688, -73.96362342)",WINTER,208259
10,731283092,12/31/2015,23:05:00,12/31/2015,23:15:00,12/31/2015,341,PETIT LARCENY,343.0,"LARCENY,PETIT OF BICYCLE",...,FAST FOOD,NaN,NaN,1014216.0,238784.0,40.822040,-73.891732,"(40.822039935, -73.891732267)",WINTER,52246
